# Derive atmospheric clear-sky and all-sky heating rates from radiation fluxes

This script diagnoses the clear-syk and all-sky heating rates so that the cloud-radiative heating rate can be calculated. It uses the function calc_heatingrates in calc_heatingrates.py (which mimicks the relevant ICON code) and stores the heating rates in a zarr store. The latter is used as it appears to that writing zarr files is faster and uses less disk space than storing the same data in netcdf files.

Import python backbone libraries:

In [1]:
import numpy as np
import xarray as xr
from pathlib import Path
import shutil
import zarr

In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:35418 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 48 Memory: 134.22 GB


Import function to calculate heating rates:

In [3]:
import calc_heatingrates

Define function for loading required data into dataset:

In [4]:
def load_data(expid, tstep):
    
    # expid is the simulation id
    # tstep is the time step as an integer, which is converted to a string with leading zeros
    
    path  = '/scratch/b/b380459/icon_4_hackathon/'
    # pressure, density, specific humidity at full levels, and height at half ("interface") levels
    fname_fg = path+'/'+expid+'/'+expid+'_2016*_fg_DOM01_ML_'+str(tstep).zfill(4)+'.nc'
    # radiative fluxes inside the atmosphere at half levels
    fname_3drad = path+'/'+expid+'/'+expid+'_2016*_3drad_DOM01_ML_'+str(tstep).zfill(4)+'.nc'
    ds_fg = ( xr.open_mfdataset(fname_fg, combine='by_coords',parallel=True, 
                                engine='h5netcdf', chunks={'ncells_2': 1e6})
             .rename({'ncells_2': 'ncells', 'height_3': 'height_ifc'})[['pres', 'qv', 'z_ifc', 'rho']] )
    ds_3drad = ( xr.open_mfdataset(fname_3drad, combine='by_coords',parallel=True, 
                                   engine='h5netcdf', chunks={'ncells': 1e6})
                .rename({'height': 'height_ifc'}) )
    
    # merge and return dataset
    return xr.merge([ds_fg, ds_3drad])
    

Calculate heating rates and store to zarr store:

For this, we are looping over the simulations manually.

In [5]:
for res in ['2']: #80', '40', '20', '10']:#, '5', '2']:
    #for nbr in ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012']:
    for nbr in [ #'0001-shcon', '0002-shcon', '0005-shcon', '0006-shcon', '0007-shcon', '0008-shcon',
                 #'0009-shcon', '0010-shcon', 
                '0011-shcon', '0012-shcon']:
        expid = 'nawdexnwp-'+res+'km-mis-'+nbr
        print('Working on:', expid)
        zarr_store = '/work/bb1018/nawdex-hackathon_pp/ddttemp_rad-from-fluxes/'+expid+'_ddttemp_rad-from-fluxes_DOM01_ML.zarr'
        # remove any zarr_store with same name that might have been created previously
        shutil.rmtree(zarr_store, ignore_errors=True)
        # create sorted list of available timesteps
        tstep_list = []
        for i in Path('/scratch/b/b380459/icon_4_hackathon/'+expid).rglob(expid+'_2016*_fg_DOM01_ML_*.nc'):
            tstep_list.append(str(i).split('/')[-1].split('_')[5].split('.')[0])
        tstep_list.sort(key=int)
        # loop over all available timesteps, provided the timestep list is not empty
        if tstep_list:
            for tstep in tstep_list:
                ds = load_data(expid, tstep)
                ds = calc_heatingrates.calc_heatingrates(ds)
                # if this is the first timestep, then create new zarr store, otherwise append to existing zarr store
                if Path(zarr_store).exists():
                    ds[['ddt_temp_radsw_fromflux', 'ddt_temp_radlw_fromflux', 
                        'ddt_temp_radswclr_fromflux', 'ddt_temp_radlwclr_fromflux']].to_zarr(zarr_store, append_dim='time')
                else:
                    ds[['ddt_temp_radsw_fromflux', 'ddt_temp_radlw_fromflux', 
                        'ddt_temp_radswclr_fromflux', 'ddt_temp_radlwclr_fromflux']].to_zarr(zarr_store)

Working on: nawdexnwp-2km-mis-0011-shcon
Working on: nawdexnwp-2km-mis-0012-shcon


## Shut down dask

In [6]:
client.close()